In [2]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from mypipes import *

In [6]:
train_file=r"E:\IIT-K\data\data\loan_data_train.csv"
test_file=r"E:\IIT-K\data\data\loan_data_test.csv"

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)

ld_train.head()


ID Amount.Requested Amount.Funded.By.Investors Interest.Rate  \
0  79542.0            25000                      25000        18.49%   
1  75473.0            19750                      19750        17.27%   
2  67265.0             2100                       2100        14.33%   
3  80167.0            28000                      28000        16.29%   
4  17240.0            24250                   17431.82        12.23%   

  Loan.Length        Loan.Purpose Debt.To.Income.Ratio State Home.Ownership  \
0   60 months  debt_consolidation               27.56%    VA       MORTGAGE   
1   60 months  debt_consolidation               13.39%    NY       MORTGAGE   
2   36 months      major_purchase                3.50%    LA            OWN   
3   36 months         credit_card               19.62%    NV       MORTGAGE   
4   60 months         credit_card               23.79%    OH       MORTGAGE   

   Monthly.Income FICO.Range Open.CREDIT.Lines Revolving.CREDIT.Balance  \
0         8606.56    720-724                11                    15210   
1         6737.50    710-714                14                    19070   
2         1000.00    690-694                13                      893   
3         7083.33    710-714                12                    38194   
4         5833.33    730-734                 6                    31061   

   Inquiries.in.the.Last.6.Months Employment.Length  
0                             3.0           5 years  
1                             3.0           4 years  
2                             1.0          < 1 year  
3                             1.0         10+ years  
4                             2.0         10+ years

In [9]:
ld_test.head()

ID  Amount.Requested Amount.Funded.By.Investors Loan.Length  \
0  20093              5000                       5000   60 months   
1  62445             18000                      18000   60 months   
2  65248              7200                       7200   60 months   
3  81822              7200                       7200   36 months   
4  57923             22000                      22000   60 months   

         Loan.Purpose Debt.To.Income.Ratio State Home.Ownership  \
0              moving               12.59%    NY           RENT   
1  debt_consolidation                4.93%    CA           RENT   
2  debt_consolidation               25.16%    LA       MORTGAGE   
3  debt_consolidation               17.27%    NY       MORTGAGE   
4  debt_consolidation               18.28%    MI       MORTGAGE   

   Monthly.Income FICO.Range Open.CREDIT.Lines  Revolving.CREDIT.Balance  \
0         4416.67    690-694                13                      7686   
1         5258.50    710-714                 6                     11596   
2         3750.00    750-754                13                      7283   
3         3416.67    790-794                14                      4838   
4         6083.33    720-724                 9                     20181   

   Inquiries.in.the.Last.6.Months Employment.Length  
0                               0          < 1 year  
1                               0         10+ years  
2                               0           6 years  
3                               0         10+ years  
4                               0           8 years

In [10]:
ld_train.sample(10)

ID Amount.Requested Amount.Funded.By.Investors Interest.Rate  \
1233  96479.0            16000                      16000        17.27%   
1529  22315.0             7000                       6875        10.00%   
433   41426.0             8400                       8400        14.27%   
1089  31004.0            25450                      25425        13.99%   
680   60064.0            20000                      20000        19.72%   
1303  39482.0             5000                       5000        12.42%   
1713  31254.0            17000                      17000        11.49%   
1128  28534.0            10000                       9975        16.49%   
554   30497.0            10000                      10000         7.49%   
369   52129.0            23000                      23000         7.62%   

     Loan.Length        Loan.Purpose Debt.To.Income.Ratio State  \
1233   60 months  debt_consolidation                8.44%    NY   
1529   36 months    home_improvement                1.68%    CO   
433    36 months  debt_consolidation               11.95%    NC   
1089   60 months         credit_card               26.38%    NC   
680    36 months  debt_consolidation               13.74%    VA   
1303   36 months  debt_consolidation               23.84%    SC   
1713   36 months  debt_consolidation               13.69%    MI   
1128   60 months         credit_card               17.70%    IL   
554    36 months  debt_consolidation                7.77%    AZ   
369    36 months                 car                6.51%    CA   

     Home.Ownership  Monthly.Income FICO.Range Open.CREDIT.Lines  \
1233           RENT         4666.67    695-699                 9   
1529           RENT         2375.00    745-749                 3   
433        MORTGAGE         4166.67    680-684                10   
1089           RENT         4416.67    720-724                13   
680        MORTGAGE         7965.00    665-669                18   
1303           RENT         2500.00    695-699                 8   
1713           RENT         5083.33    695-699                 9   
1128       MORTGAGE         5416.67    705-709                10   
554            RENT         3166.67    735-739                 4   
369        MORTGAGE        25000.00    810-814                10   

     Revolving.CREDIT.Balance  Inquiries.in.the.Last.6.Months  \
1233                    15200                             0.0   
1529                      922                             0.0   
433                     16634                             1.0   
1089                    31828                             2.0   
680                     18809                             1.0   
1303                     7980                             2.0   
1713                    13883                             0.0   
1128                    41329                             0.0   
554                         0                             0.0   
369                       227                             0.0   

     Employment.Length  
1233         10+ years  
1529          < 1 year  
433          10+ years  
1089           3 years  
680            7 years  
1303           2 years  
1713           4 years  
1128         10+ years  
554             1 year  
369          10+ years

In [11]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [12]:
ld_train["Loan.Length"].value_counts(dropna=False)

36 months    1722
60 months     476
NaN             1
.               1
Name: Loan.Length, dtype: int64